In [1]:
# import the SimPy package 
#from simpy import initialize
#from SimPy.Simulation import *
import simpy
import random
import collections

In [50]:
def distribucion_llegada(env,nClientes,puestos,cont):
    for i in range(nClientes):
        r = random.random()
        c = Customer(name = "clienteE%02d"%i)
        if r <= cont["PE"]:
            ia = random.uniform(cont["min"],cont["max"])
            yield env.timeout(ia)
            yield env.process(c.llamada_emergencia(env,puestos,cont))
        else: 
            ia = random.uniform(cont["min"],cont["max"])
            yield env.timeout(ia)
            #print(puestos)
            yield env.process(c.llamada_comercial(env,puestos,cont))

In [48]:
class Customer():
    """Evento llegadas/llamadas de los clientes"""
    def __init__(self,name):
        self.name = name
        
    def llamada_emergencia(self,env,puestos,cont):
        arrive = env.now
        # Pido uso de los dos puestos, y me dan el primero que se libere
        comercial = puestos[0].request(priority=0)
        emergencia = puestos[1].request(priority=0)
        atendido = yield comercial | emergencia
        
        # Me atendieron por puestos comerciales?
        if comercial in atendido:
            cont["multitasking"]+=1
            wait = env.now-arrive #Tiempo de espera
            print(env.now,self.name,": EMERGENCIA - Me atendieron en un puesto comercial despues de esperar ",wait)
            ta= abs(random.normalvariate(cont["mu"],cont["sigma"]))
            yield env.timeout(ta)
            
        else:
            wait = env.now-arrive #Tiempo de espera
            #wM.observe(wait) #Monitor de espera (sumatoria de espera en cola)
            #print "%7.4f %s: EMERGENCIA - Me atendieron en el puesto de emergencia despues de esperar %6.3f"%(env.now,self.name,wait)
            print(env.now,self.name,": EMERGENCIA - Me atendieron en un puesto de emergencia despues de esperar ",wait)
            #print(env.now,": EMERGENCIA - Me atendieron en un puesto de emergencia despues de esperar ",wait)
            ta= abs(random.normalvariate(cont["mu"],cont["sigma"]))
            yield env.timeout(ta)
            
            
    def llamada_comercial(self,env,puestos,cont):
        arrive = env.now
        #print
        
        # Pido uso de los dos puestos, y me dan el primero que se libere
        with puestos[0].request(priority=1) as request:
            yield request
            
            wait = env.now-arrive #Tiempo de espera
            print(env.now,self.name,": COMERCIAL - Me atendieron despues de esperar ",wait)
            ta= abs(random.normalvariate(cont["mu"],cont["sigma"]))
            yield env.timeout(ta)
        

Condiciones Iniciales

In [4]:
# Experimental data
maxNumber = 100 #clientes
maxTime = 14400 #segundos
#random.seed(RANDOM_SEED)
PE = 0.12 #Porcentaje Emergencia
PC = 0.88 #Porcentaje Comercial

# TA - distribucion normal
mu = 267.64 
sigma = 297.40

# IA - uniforme
IAMin = 1
IAMax = 60

# Contadores
ci = {"multitasking":0,
     "mu":mu,
     "sigma":sigma,
     "min":IAMin,
     "max":IAMax,
     "PE":PE,
     "PC":PC}

# Puestos de atencion - variables de control
print('Ingrese cantidad de puestos comerciales')
cantCom = int(input())
print('Ingrese cantidad de puestos de emergencia')
cantEmer = int(input())

Ingrese cantidad de puestos comerciales
12
Ingrese cantidad de puestos de emergencia
11


Modelo de Simulacion

In [51]:
env = simpy.Environment()
puestos = [simpy.PriorityResource(env,capacity=cantCom),
           simpy.PriorityResource(env,capacity=cantEmer)]
           # menor el numero priority, mayor es la prioridad
#wM = Monitor()
#s = Source(env,maxNumber,puestos,ci)
env.process(distribucion_llegada(env,maxNumber,puestos,ci))

env.run(until=maxTime)

8.11017984175319 clienteE00 : COMERCIAL - Me atendieron despues de esperar  0.0
628.6892833510326 clienteE01 : COMERCIAL - Me atendieron despues de esperar  0.0
735.2163004038663 clienteE02 : EMERGENCIA - Me atendieron en un puesto comercial despues de esperar  0.0
1505.1843952317631 clienteE03 : COMERCIAL - Me atendieron despues de esperar  0.0
1965.9569438343158 clienteE04 : COMERCIAL - Me atendieron despues de esperar  0.0
2639.365058472099 clienteE05 : COMERCIAL - Me atendieron despues de esperar  0.0
3218.2253988363937 clienteE06 : COMERCIAL - Me atendieron despues de esperar  0.0
3276.2327204371263 clienteE07 : COMERCIAL - Me atendieron despues de esperar  0.0
3544.208935087059 clienteE08 : COMERCIAL - Me atendieron despues de esperar  0.0
3628.963654841875 clienteE09 : COMERCIAL - Me atendieron despues de esperar  0.0
3913.87463143638 clienteE10 : COMERCIAL - Me atendieron despues de esperar  0.0
4651.716796177811 clienteE11 : COMERCIAL - Me atendieron despues de esperar  0.0
51

Results

In [ ]:
#result = wM.count(),wM.mean() #wM.mean --> promedio de espera en cola
print "Average wait for %3d completions was %5.3f minutes."% result

# Porcentaje de espera menor a 40 segundos
# Objetio: Mayor o igual al 90%

print('Cantidad de puestos comerciales',cantCom)
print('Cantidad de puestos de emergencia',cantEmer)

In [42]:
print(maxNumber)

100
